<a href="https://colab.research.google.com/github/MariaInyutina/Parcing/blob/main/Parcing_extra_TG_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telethon

In [ ]:
from telethon import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from datetime import datetime, timedelta, timezone
import pandas as pd

# Параметры для Telegram API
API_ID =
API_HASH = ''
CHANNEL_USERNAME = ''  # Имя пользователя канала
phone = ''
session_name = 'parcing'
# Инициализация клиента
client = TelegramClient(session_name, API_ID, API_HASH)

# Функция для сбора статистики за определённый период
async def get_channel_stats(start_date, end_date):
    await client.start()
    stats = []

    # Получение истории сообщений канала за указанный период
    async for message in client.iter_messages(CHANNEL_USERNAME, offset_date=end_date):
        if message.date < start_date:
            break

        # Подсчет количества реакций
        reactions_count = sum(reaction.count for reaction in message.reactions.results) if message.reactions else 0

        # Сбор информации о сообщении
        data = {
            'date': message.date.strftime('%d-%m-%y'),  # Форматируем дату как ДД-ММ-ГГ
            'post_title': message.message[:30] if message.message else "Без текста",  # Первые 30 символов текста как заголовок
            'post_link': f'https://t.me/{CHANNEL_USERNAME}/{message.id}',  # Ссылка на пост
            'views': message.views,
            'reactions': reactions_count,
            'comments': message.replies.replies if message.replies else 0,
            'public_reposts': message.forwards if message.forwards else 0,
            'private_reposts': None,  # Telegram не предоставляет разделение репостов на публичные и непубличные
            'comments': message.replies.replies if message.replies else 0
        }
        stats.append(data)

    # Сохранение данных в таблицу
    df = pd.DataFrame(stats)
    df.to_excel('telegram_extra_stats.xlsx', index=False)  # Сохранение в файл Excel
    print("Данные успешно сохранены в telegram_extra_stats.xlsx")

# Запрос ввода даты от пользователя
start_date_input = input("Введите начальную дату в формате ГГГГ-ММ-ДД: ")
end_date_input = input("Введите конечную дату в формате ГГГГ-ММ-ДД: ")

# Преобразование ввода в формат даты с временной зоной UTC
start_date = datetime.strptime(start_date_input, '%Y-%m-%d').replace(tzinfo=timezone.utc)
end_date = datetime.strptime(end_date_input, '%Y-%m-%d').replace(tzinfo=timezone.utc)

# Основная функция с использованием async with
async def main():
    async with client:
        await get_channel_stats(start_date, end_date)

# Запуск main() в асинхронной среде
await main()

**Использовать следующую ячейку при многоразовом запуске кода, нужна для завершения сеанса**

In [ ]:
# Завершение клиента и пауза
await client.disconnect()
#await asyncio.sleep(1)  # Небольшая пауза для завершения фоновых задач

**Посмотреть получившуюсь таблицу**

In [ ]:
df = pd.read_excel('telegram_extra_stats.xlsx')
df

# **Для ежемесячной статистики**:

**Скачиваем финальную таблицу**

In [ ]:
df_transposed.to_excel('telegram_extra_stats.xlsx', index=True)
from google.colab import files

# Скачивание файла
files.download('telegram_extra_stats.xlsx')

# **Для еженедельной статистики**

In [ ]:
# Переставляем столбцы в нужном порядке
df = df[['post_title', 'post_link', 'date', 'views', 'reactions', 'public_reposts', 'private_reposts', 'comments']]

# Преобразуем столбец с датой в формат ДД-ММ
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%y').dt.strftime('%d-%m')

# Транспонируем DataFrame, чтобы сделать столбцы строками
df_transposed = df.T

# Переименовываем индексы в соответствии с требуемыми названиями
df_transposed.index = [
    'Название поста',
    'Ссылка',
    'Дата',
    'Просмотры',
    'Реакции',
    'Репосты публичные',
    'Репосты непубличные',
    'Комментарии'
]
# Теперь df_transposed содержит нужную структуру
df_transposed

**Скачиваем финальную таблицу**

In [ ]:
df_transposed.to_excel('telegram_extra_stats.xlsx', index=True)
from google.colab import files

# Скачивание файла
files.download('telegram_extra_stats.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>